## Objetivo:

Predecir la probabilidad de que un individuo tweetee en un tweet positivo o negativo dependiendo del día de la semana.

NO SE REALMENTE, NO ESTOY TAN SEGURO EN BASE AL GRÁFICO.



_______________________________________

In [ ]:
# Librerías
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from scipy.stats import mode
import seaborn as sns
import re

# nltk
import nltk
from nltk.corpus import stopwords
from  nltk.stem import SnowballStemmer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV,StratifiedKFold,train_test_split
from sklearn.metrics import accuracy_score

from sklearn.pipeline import Pipeline

## Importación

BAJAR DATASET DE KAGGLE, RENOMBRARLO COMO TWITTER Y PONERLO EN LA CARPETA DATA

In [ ]:
data = pd.read_csv('./data/twitter.csv', encoding='ANSI')
data.columns = [ 'target', 'id', 'date', 'flag', 'user', 'text']
print(data.shape)
data.head()

In [ ]:
# Get empty values
data.isnull().sum()


In [ ]:
# Set day of week as category
data['date'] = data['date'].str[:3].astype('category')
data['date'].cat.categories = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
data.head()


In [ ]:
# Get the number of tweets per day and target
data.groupby(['date', 'target']).size()

In [ ]:
# Plot the number of tweets per day and target with a bar chart
# {0: 'Negative', 2: 'Neutral', 4: 'Positive'}
data.groupby(['date', 'target']).size().unstack().plot(kind='bar',  figsize=(10, 5))
plt.title('Number of tweets per day and target')
plt.xlabel('Day of week')
plt.ylabel('Number of tweets')
plt.show()



## Preprocesamiento de texto

In [ ]:
# para la limpieza de texto
limpieza_re = "@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+"

In [ ]:
stop_words = stopwords.words("english")
stemmer = SnowballStemmer("english")

In [ ]:
#Funcion para limpiar el corpus
def preprocess(text, stem=False):
    # Remove link,user and special characters
    text = re.sub(limpieza_re, ' ', str(text).lower()).strip()
    tokens = []
    for token in text.split():
        if token not in stop_words:
            if stem:
                tokens.append(stemmer.stem(token))
            else:
                tokens.append(token)
    return " ".join(tokens)

In [ ]:
data.text = data.text.apply(lambda x: preprocess(x))

In [ ]:
data.text

In [ ]:
train,test=train_test_split(data,stratify=data['target'],random_state=3);

In [ ]:
# train,test,y_train,y_test=train_test_split(data["text"],data['target']);

In [ ]:
# vectorizer=TfidfVectorizer();
# X_train=vectorizer.fit_transform(train['text']);
# y_train=train['target'];

# X_test=vectorizer.transform(test['text']);
# y_test=test['target'];

In [ ]:
vectorizer=TfidfVectorizer(stop_words=stop_words,strip_accents='unicode');
X_train=vectorizer.fit_transform(train['text']);
y_train=train['target'];

X_test=vectorizer.transform(test['text']);
y_test=test['target'];



In [ ]:
skf=StratifiedKFold(n_splits=3,random_state=0,shuffle=True)

param_grid={'alpha':np.arange(0.05,1,0.05)};
grid = GridSearchCV(MultinomialNB(), param_grid, cv=skf,verbose=1);
grid.fit(X_train, y_train);
print("Best cross-validation score: {:.4f}".format(grid.best_score_));
print("Best parameters: ", grid.best_params_);

In [ ]:

model=Pipeline([('vect',vectorizer),('classifier',MultinomialNB())])

params={'classifier__alpha':[0.1,0.5,1],'vect__min_df':[1,3]};

GS_CV=GridSearchCV(model,params,cv=skf,verbose=1,n_jobs=-1);

GS_CV=GridSearchCV(MultinomialNB(),params,cv=skf,verbose=1,n_jobs=3,refit=False)

GS_CV.fit(train, y_train);

print('best score:',GS_CV.best_score_)
print('best params:',GS_CV.best_params_)

### Wordcloud

### Tokenización

### Remoción de stopwords

### Lematización/ Stemming

### Vectorización

### TF-IDF

### Word2Vec

### Singular Value Decomposition 

## Modelos